# 3 Linear Regression - curve fitting (TensorFlow)
Two methods are used to fit a curve:
- Direct solution using least-squares method
- Iterative optimisation using stochastic gradient descent

## 3.1 Data

In [1]:
import random
import numpy as np

# get ground-truth data from the "true" model 
n = 100 
w = [4, 3, 2, 1]
x = np.linspace(-1,1,n)[:,np.newaxis]
t = np.matmul(np.power(np.reshape(x,[-1,1]),np.linspace(len(w)-1,0,len(w))), w)[:,np.newaxis]
std_noise = 0.2
t_observed = [t[idx]+random.gauss(0,std_noise) for idx in range(n)]

## 3.2 Computation Graph and Session

In [2]:
import tensorflow as tf

# placeholders are for feeding data in runtime
ph_x = tf.placeholder(tf.float32, [n, 1])
ph_t = tf.placeholder(tf.float32, [n, 1])

deg = 3
node_X = tf.pow(ph_x, tf.linspace(tf.to_float(deg),0,deg+1))

# build a session
sess = tf.Session()  

# set an example data feed
dataFeed = {ph_x:x} 

# run the session to evaluate the node weights
X = sess.run(node_X, feed_dict=dataFeed)
print(X[:n:10,])

sess.close()

[[-1.00000000e+00  1.00000000e+00 -1.00000000e+00  1.00000000e+00]
 [-5.08130968e-01  6.36771739e-01 -7.97979772e-01  1.00000000e+00]
 [-2.11665690e-01  3.55167836e-01 -5.95959604e-01  1.00000000e+00]
 [-6.11347705e-02  1.55188262e-01 -3.93939406e-01  1.00000000e+00]
 [-7.06895534e-03  3.68329771e-02 -1.91919193e-01  1.00000000e+00]
 [ 1.03061018e-06  1.02030404e-04  1.01010101e-02  1.00000000e+00]
 [ 9.54448152e-03  4.49954122e-02  2.12121218e-01  1.00000000e+00]
 [ 7.10306838e-02  1.71513110e-01  4.14141417e-01  1.00000000e+00]
 [ 2.33928949e-01  3.79655182e-01  6.16161644e-01  1.00000000e+00]
 [ 5.47708452e-01  6.69421494e-01  8.18181813e-01  1.00000000e+00]]


## 3.3 Least-squares Solution

In [3]:
# completing the computation graph with the least-square solution
node_w = tf.matrix_solve_ls(node_X, ph_t)

# run the session to evaluate the node weights
sess = tf.Session()  
dataFeed = {ph_x:x, ph_t:t_observed}  # feed data
w_lstsq = sess.run(node_w, feed_dict=dataFeed)
print(w_lstsq)

sess.close()

[[3.9082298]
 [2.9749825]
 [2.0859468]
 [1.0166769]]


## 3.3 Stochastic Gradient Descend Method
Instead of least-squares, weights can be optimised by minimising a loss function between the predicted- and observed target values, using SGD. This is for demo purpose.

In [4]:
# build a new graph
ph_1x = tf.placeholder(tf.float32, [1, 1])
ph_1t = tf.placeholder(tf.float32, [1, 1])

deg = 3
node_X = tf.pow(ph_1x, tf.linspace(tf.to_float(deg),0,deg+1))

# first declare variables that need optimisation
var_w = tf.get_variable('weights', shape=[deg+1,1], 
                        initializer=tf.random_normal_initializer(0, 0.01))

# completing the computation graph with SGD
node_1t = tf.matmul(node_X, var_w)
# building a square loss
loss = tf.reduce_mean(tf.square(node_1t-ph_1t))
# buiding a train-op to minimise the loss
train_op = tf.train.GradientDescentOptimizer(learning_rate=1e-2).minimize(loss)

# run the session to evaluate the node weights
sess = tf.Session()  
sess.run(tf.global_variables_initializer())  # initialise all the variables

# iteration to update variables with backprop gradients
total_iter = int(1e3)
indices_train = [i for i in range(n)]
for step in range(total_iter):

    idx = step % n
    if (idx == 0):
        random.shuffle(indices_train)
    
    # single data point feed
    singleDataFeed = {
        ph_1x:np.reshape(x[indices_train[idx]],[1,1]), 
        ph_1t:np.reshape(t_observed[indices_train[idx]],[1,1]) }  # feed data
    
    # update the variables
    sess.run(train_op, feed_dict=singleDataFeed)
    
    # print training information
    loss_train, w_sgd = sess.run([loss,var_w], feed_dict=singleDataFeed)
    print('Step %d: Loss=%f' % (step, loss_train))
    # print('Estimated weights=%f' % w_sgd)

print(w_sgd)
sess.close()

Step 0: Loss=1.610301
Step 1: Loss=2.286861
Step 2: Loss=0.187722
Step 3: Loss=4.796022
Step 4: Loss=0.196458
Step 5: Loss=0.022310
Step 6: Loss=0.297468
Step 7: Loss=0.073123
Step 8: Loss=0.002630
Step 9: Loss=0.015601
Step 10: Loss=45.657604
Step 11: Loss=42.924362
Step 12: Loss=16.544363
Step 13: Loss=9.212370
Step 14: Loss=0.199671
Step 15: Loss=0.794427
Step 16: Loss=0.429262
Step 17: Loss=1.875351
Step 18: Loss=39.260822
Step 19: Loss=0.047531
Step 20: Loss=2.702766
Step 21: Loss=4.524525
Step 22: Loss=0.023216
Step 23: Loss=1.235247
Step 24: Loss=0.002523
Step 25: Loss=0.061835
Step 26: Loss=40.447315
Step 27: Loss=0.069833
Step 28: Loss=0.417017
Step 29: Loss=53.943371
Step 30: Loss=0.433590
Step 31: Loss=0.295877
Step 32: Loss=1.164807
Step 33: Loss=9.956680
Step 34: Loss=8.386155
Step 35: Loss=0.148355
Step 36: Loss=1.799938
Step 37: Loss=0.474947
Step 38: Loss=1.046067
Step 39: Loss=0.207519
Step 40: Loss=3.474642
Step 41: Loss=0.016646
Step 42: Loss=3.293655
Step 43: Loss=0

Step 456: Loss=0.000254
Step 457: Loss=0.043075
Step 458: Loss=0.134795
Step 459: Loss=0.137424
Step 460: Loss=0.040709
Step 461: Loss=0.230069
Step 462: Loss=0.000673
Step 463: Loss=0.002823
Step 464: Loss=0.091655
Step 465: Loss=0.161359
Step 466: Loss=0.328337
Step 467: Loss=0.008237
Step 468: Loss=0.413308
Step 469: Loss=0.428666
Step 470: Loss=0.003631
Step 471: Loss=0.684699
Step 472: Loss=0.143677
Step 473: Loss=0.062461
Step 474: Loss=0.005433
Step 475: Loss=0.040197
Step 476: Loss=0.001542
Step 477: Loss=0.022326
Step 478: Loss=0.006242
Step 479: Loss=0.044240
Step 480: Loss=0.039370
Step 481: Loss=0.187155
Step 482: Loss=0.127492
Step 483: Loss=0.000319
Step 484: Loss=0.071658
Step 485: Loss=0.349090
Step 486: Loss=0.000843
Step 487: Loss=0.000458
Step 488: Loss=0.003977
Step 489: Loss=0.045567
Step 490: Loss=0.002155
Step 491: Loss=0.002268
Step 492: Loss=0.404439
Step 493: Loss=0.070497
Step 494: Loss=0.071756
Step 495: Loss=0.091995
Step 496: Loss=0.109044
Step 497: Loss=0

Step 933: Loss=0.035576
Step 934: Loss=0.005743
Step 935: Loss=0.535105
Step 936: Loss=0.032585
Step 937: Loss=0.000139
Step 938: Loss=0.103847
Step 939: Loss=0.136569
Step 940: Loss=0.189650
Step 941: Loss=0.020779
Step 942: Loss=0.330156
Step 943: Loss=0.226219
Step 944: Loss=0.005101
Step 945: Loss=0.000385
Step 946: Loss=0.085622
Step 947: Loss=0.016185
Step 948: Loss=0.042076
Step 949: Loss=0.015127
Step 950: Loss=0.006395
Step 951: Loss=0.021092
Step 952: Loss=0.064309
Step 953: Loss=0.078696
Step 954: Loss=0.011157
Step 955: Loss=0.076791
Step 956: Loss=0.006401
Step 957: Loss=0.123920
Step 958: Loss=0.000801
Step 959: Loss=0.047968
Step 960: Loss=0.013425
Step 961: Loss=0.194818
Step 962: Loss=0.109883
Step 963: Loss=0.311489
Step 964: Loss=0.014038
Step 965: Loss=0.278951
Step 966: Loss=0.092113
Step 967: Loss=0.039628
Step 968: Loss=0.295140
Step 969: Loss=0.050042
Step 970: Loss=0.093008
Step 971: Loss=0.038263
Step 972: Loss=0.283831
Step 973: Loss=0.046569
Step 974: Loss=0